# Hatodik gyakorlat - LP Solverek (PuLP)

In [3]:
import pulp as pl
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

In [ ]:
!pip install pulp

# Modellezés PuLP-pal

Modell létrehozása PuLP-pal:

In [33]:
prob = pl.LpProblem(name="ProblemName",
                    sense=pl.LpMaximize) # vagy pl.LpMinimize (ez a default)

Változók létrehozása:

In [28]:
x = pl.LpVariable(name="x",
                  lowBound=1,
                  upBound=10,
                  cat=pl.LpInteger) # vagy pl.LpContinuous vagy pl.LpBinary

Változók létrehozás dictionary-ként:

In [29]:
idx = ["I1", "I2", "A1", "A2"]
y = pl.LpVariable.dicts(name="y",
                        indexs=idx,
                        cat=pl.LpBinary)

In [30]:
y

{'I1': y_I1, 'I2': y_I2, 'A1': y_A1, 'A2': y_A2}

Korlátok hozzáadása a modellhez:

In [34]:
prob += (x + y["I1"] <= 2, "elso constraint")
prob += pl.lpSum(y) == 3, "masodik constraint"
prob += x + y["A1"] >= 3

Célfüggvény hozzáadása a modellhez:

In [35]:
cost = {
    "I1" : 5,
    "I2" : 2,
    "A1" : 3,
    "A2" : 4
}
prob += pl.lpSum(y[i] * cost[i] for i in idx)

Modell kiírása:

In [36]:
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

_C1: x + y_A1 >= 3

VARIABLES
1 <= x <= 10 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Modell megoldása:

In [48]:
prob.solve(pl.PULP_CBC_CMD(msg=True))

1

Probléma állapotok:

In [39]:
pl.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

Megoldás "megszerzése":

In [41]:
print("x=",pl.value(x))
for i in idx:
    print(y[i].name, "=", pl.value(y[i]))

x= 2.0
y_I1 = 0.0
y_I2 = 1.0
y_A1 = 1.0
y_A2 = 1.0


Célfüggvényérték:

In [42]:
pl.value(prob.objective)

9.0

Korlátok ellenőrzése:

Azt mutatja, hogy például egy $ax\leq b$ korlát esetén mennyi az $ax-b$ értéke. Ezeknek egyenlőség esetén nyilván $0$-nak kell lenni.

In [43]:
print(pl.value(prob.constraints["_C1"]))
print(pl.value(prob.constraints["elso_constraint"]))
print(pl.value(prob.constraints["masodik_constraint"]))

0.0
0.0
0.0


Korlát törlése:

In [44]:
del prob.constraints["_C1"]
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
1 <= x <= 10 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Változó korlátok módosítása:

In [45]:
x.lowBound=2
x.upBound=4
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
2 <= x <= 4 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Változó típusának módosítása:

In [46]:
y["A1"].cat = pl.LpInteger
y["A1"].lowBound = 0
y["A1"].upBound = 3
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
2 <= x <= 4 Integer
0 <= y_A1 <= 3 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Ha a `%%python` cellamágiát használjátok, akkor az optimalizálási folyamat logját is bizotsan visszakapjátok.

In [49]:
%%python
import pulp as pl
prob = pl.LpProblem(name="ProblemName",
                    sense=pl.LpMaximize) # vagy pl.LpMinimize (ez a default)

x = pl.LpVariable(name="x",
                  lowBound=1,
                  upBound=10,
                  cat=pl.LpInteger) # vagy pl.LpContinuous vagy pl.LpBinary

idx = ["I1", "I2", "A1", "A2"]
y = pl.LpVariable.dicts(name="y",
                        indexs=idx,
                        cat=pl.LpBinary)

prob += x + y["I1"] <= 2, "elso constraint"
prob += pl.lpSum(y) == 3, "masodik constraint"
prob += x + y["A1"] >= 3

cost = {
    "I1" : 5,
    "I2" : 2,
    "A1" : 3,
    "A2" : 4
}
prob += pl.lpSum(y[i] * cost[i] for i in idx)

prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - C:\Users\Peter Dobrovoczki\anaconda3\envs\notebook\lib\site-packages\pulp\apis\..\solverdir\cbc\win\64\cbc.exe C:\Users\PETERD~1\AppData\Local\Temp\077ee00e91804b20bbe69fb8fb1671fe-pulp.mps max timeMode elapsed branch printingOptions all solution C:\Users\PETERD~1\AppData\Local\Temp\077ee00e91804b20bbe69fb8fb1671fe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 31 RHS
At line 35 BOUNDS
At line 42 ENDATA
Problem MODEL has 3 rows, 5 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -9 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active

Elérhető solverek:

In [51]:
pl.listSolvers(onlyAvailable=True)

Set parameter Username


['GLPK_CMD', 'XPRESS', 'PULP_CBC_CMD']

# Feladatok

## Keverési feladat

Szeretnénk kikeverni $25$ tonna speciális összetételű acélt különböző acéltömbökből. A kapott keveréknek az $5\%$-a legyen szén és $5\%$-a molibdén. Négy féle acél áll rendelkezésünkre, mindegyiket tömb formájában lehet megvenni, és mindegyikből legfeljebb $1$-et. A tömbök összetétele:

| Tömb | Súly (tonna) | Szén (%) | Molibdén (%) | Ár (\$/tonna) |
| ---- | ------------ | -------- | ------------ | ------------- |
| 1    | 5            | 5        | 3            | 350           |
| 2    | 3            | 4        | 3            | 330           |
| 3    | 4            | 5        | 4            | 310           |
| 4    | 6            | 3        | 4            | 280           |

Ezen kívül megvásárolható $3$ féle ötvözet is, illetve hulladékfémet is tudunk venni. Ezekből törtmennyiségeket is lehet vásárolni. Az összetételük:

| Ötvözet  | Szén (%) | Molibdén (%) | Ár (\$/tonna) |
| -------- | -------- | ------------ | ------------- |
| 1        | 8        | 6            | 500           |
| 2        | 7        | 7            | 450           |
| 3        | 6        | 8            | 400           |
| Hulladék | 3        | 9            | 100           |

A cél a lehető legolcsóbban előállítani a kívánt ötvözetet.

In [20]:
# Problem data
carbon = {
    "I1" : 0.05,
    "I2" : 0.04,
    "I3" : 0.05,
    "I4" : 0.03,
    "A1" : 0.08,
    "A2" : 0.07,
    "A3" : 0.06,
    "S"  : 0.03
}

molybdenium = {
    "I1" : 0.03,
    "I2" : 0.03,
    "I3" : 0.04,
    "I4" : 0.04,
    "A1" : 0.06,
    "A2" : 0.07,
    "A3" : 0.08,
    "S"  : 0.09
}

weight = {
    "I1" : 5,
    "I2" : 3,
    "I3" : 4,
    "I4" : 6
}


cost = {
    "I1" : 350,
    "I2" : 330,
    "I3" : 310,
    "I4" : 280,
    "A1" : 500,
    "A2" : 450,
    "A3" : 400,
    "S"  : 100
}

## Max-cut
Készíts egy random gráfot a `networkx` csomaggal (`networkx.erdos_renyi_graph(...)`), és keresd meg benne a maximális vágást egy egészértékű programmal. Az IP amit meg kellene oldani:

\begin{equation}
    \begin{array}{rrclll}
        \max & \sum_{uv\in E} z_{uv} \\
        \mathrm{s.t.} & z_{uv} & \leq & x_u + x_v &\forall uv\in E\\
                      & z_{uv} & \leq & 2 - (x_u + x_v) &\forall uv\in E\\
                      & z_{uv} & \in  & \left\{0,1\right\} & \forall uv\in E\\
                      & x_v    & \in  & \left\{0,1\right\} & \forall v\in V.
    \end{array}
\end{equation}

_(Tipp: kicsi gráfokon kezdjetek kísérletezni, a PuLP által alapértelmezetten használt solver $|V|=40$ és $p=0.3$ mellett már egy percig is eltarthat.)_

__Bónusz:__ rajzold is ki a gráfot a vágáséleket kiemelve!

## Sudoku
Készítsd el a Sudoku feladat IP modelljét és oldd meg! A megoldást vizualizáld! 

__Bónusz:__ oldd meg a világ legnehezebb sudokuját: https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku

## Legkisebb abszolút eltérés

Adottak $(x_1,y_1),(x_2,y_2),\dots,(x_n,y_n)$ pontok, és szeretnék rájuk illeszteni egy egyenest, amelynek a lehető legkisebb az abszolút eltérése a pontoktól. Vagyis keressük azt az $a$ együtthatót, ami minimalizálja a

$$
    \sum_{i=1}^n \left|a x_i-y_i-b\right|
$$

összeget. Modellezd LP-ként. Generálj random pontokat és oldd meg a feladatot. Ábrázold a kapott regressziós egyenest és a pontokat.

_Hint: nyilván valami trükköt kell alkalmazni, mivel az abszolutérték függvény nem lineáris._

## Bútorlap-szabászat

Tegyük fel, hogy van egy $W\times H$ méretű téglalap alapú bútorlapom, és $n$ darab kisebb darabot szeretnék kivágni belőle. Az $i$. darab $w_i\times h_i$ méretű téglalap. Készíts el egy modellt, amellyel el lehet dönteni, hogy ki tudom-e vágni a bútorlapokat.

Legyen most $W=3\mathrm{m}$, $H=4\mathrm{m}$ és legyen 4 darab, amit ki akarunk vágni:

| Munkadarab | Szélesség (m) | Magasság (m) |
| ---------- | ------------- | ------------ |
| i1         | 1             | 3            |
| i2         | 2             | 1            |
| i3         | 2             | 2            |
| i4         | 3             | 1            |

Ki tudjuk-e vágni őket a bútorlapból, és ha igen, hogyan?
__Bónusz:__ rajzold is ki az elrendezést!